In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
import matplotlib
matplotlib.rc('image', cmap='Greys')
from fastai.vision.all import *
from fastbook import *



# End sidebar
  - For this initial tutorial we are just going to try to create a model that can classify any image as a 3 or a 7. So let's download a sample of MNIST that contains images of just these digits:

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.Base_path = path
path

In [ ]:
# path.ls?
# path.ls??
# doc(path.ls)
path.ls()

In [ ]:
(path/'train').ls()

In [ ]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

In [ ]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

In [ ]:
array(im3)[4:10, 4:10]

In [ ]:
# tensor: pythorch version numpy array
# teensor can be computed on GPU
tensor(im3)[4:10, 4:10]

In [ ]:
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:15, 4:22])
df.style.set_properties(**{'font-size': '6pt'}).background_gradient('Greys')

In [ ]:
tensor(Image.open(threes[4]))

# First try: pixel similarity


In [ ]:
three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]
len(three_tensors), len(seven_tensors)
three_tensors

In [ ]:
show_image(three_tensors[3])

In [ ]:
three_tensors[1].shape

In [ ]:
type(three_tensors)

In [ ]:
torch.stack(three_tensors)

In [ ]:
stacked_threes = torch.stack(three_tensors).float()/ 255
stacked_sevens = torch.stack(seven_tensors).float()/ 255
stacked_threes.shape, stacked_threes.ndim


In [ ]:
mean3 = stacked_threes.mean(0)
show_image(mean3)

In [ ]:
a_3 = stacked_threes[1]
show_image(a_3)

In [ ]:
mean7 = stacked_sevens.mean(0)
show_image(mean7)

- Take the mean of the absolute value of differences (absolute value is the function that replaces negative values with positive values). This is called the mean absolute difference or L1 norm

-  Take the mean of the square of differences (which makes everything positive) and then take the square root (which undoes the squaring). This is called the root mean squared error (RMSE) or L2 norm.


In [ ]:
dist_3_abs = (a_3 - mean3).abs().mean()
dist_3_sqr = ((a_3 - mean3) **2).mean().sqrt()
dist_3_abs, dist_3_sqr

In [ ]:
dist_7_abs = (a_3 - mean7).abs().mean()
dist_7_sqr = ((a_3 - mean7)** 2).mean().sqrt()
dist_7_abs, dist_7_sqr

In [ ]:
# PyTorch already provides both of these as loss functions. You'll find these inside torch.nn.functional, which the PyTorch team recommends importing as F (and is available by default under that name in fastai):
F.l1_loss(a_3.float(), mean7), F.mse_loss(a_3, mean7).sqrt()

# NumPy Arrays and PyTorch Tensors

In [ ]:
data = [[1,2,3], [4,5,6]]
arr = array(data)
tns = tensor(data)

In [ ]:
arr # numpy

In [ ]:
tns # pytorch

# Computing metrics using broadcasting

In [ ]:
valid_3_tens = torch.stack([tensor(Image.open(o)) for o  in (path/'valid'/'3').ls()])

valid_3_tens = valid_3_tens.float()/ 255

valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])

valid_7_tens = valid_7_tens.float()/ 255
valid_3_tens.shape, valid_7_tens.shape

In [ ]:
def mnist_distance(a, b): return (a- b).abs().mean((-1, -2))
mnist_distance(a_3, mean3)

In [ ]:
valid_3_dist = mnist_distance(valid_3_tens, mean3)
valid_3_dist, valid_3_dist.shape

In [ ]:
(valid_3_tens-mean3).shape

In [ ]:
def is_3(x): return mnist_distance(x,mean3) < mnist_distance(x,mean7)

In [ ]:
is_3(a_3), is_3(a_3).float()

In [ ]:
is_3(valid_3_tens)

In [ ]:
accuracy_3s =      is_3(valid_3_tens).float() .mean()
accuracy_7s = (1 - is_3(valid_7_tens).float()).mean()

accuracy_3s,accuracy_7s,(accuracy_3s+accuracy_7s)/2

# Stochastic Gradient Descent (SGD)


In [ ]:
def f(x): return x**2

In [ ]:
#id gradient_descent
# caption The gradient descent process
# alt Graph showing the steps for Gradient Descent
gv('''
init->predict->loss->gradient->step->stop
step->predict[label=repeat]
''')

In [ ]:
xt = tensor(3.).requires_grad_()

In [ ]:
yt = f(xt)
yt

In [ ]:
yt.backward()

In [ ]:
xt.grad

# Stepping with a learning rate
  - w -= gradient(w) * lr 



## An end-to-end SGD example

In [ ]:
time = torch.arange(0, 20).float()
time

In [ ]:
speed = torch.rand(20)* 3 + 0.75* (time- 9.5) **2 +1
plt.scatter(time, speed)

In [ ]:
def f(t, params): 
  a, b, c = params
  return a*(t**2) + (b*t) + c

In [ ]:
def mse(preds, targets): return ((preds- targets)**2).mean()

In [ ]:
params = torch.randn(3).requires_grad_()

In [ ]:
orig_params = params.clone()

In [ ]:
preds = f(time, params)

In [ ]:
def show_preds(preds, ax=None):
    if ax is None: ax=plt.subplots()[1]
    ax.scatter(time, speed)
    ax.scatter(time, to_np(preds), color='red')
    ax.set_ylim(-300,100)

In [ ]:
show_preds(preds)

In [ ]:
loss = mse(preds, speed)
loss

In [ ]:
loss.backward()
params.grad

In [ ]:
params.grad * 1e-5

In [ ]:
params

In [ ]:
lr = 1e-5
params.data-= lr* params.grad.data
params.grad = None

In [ ]:
preds = f(time,params)
mse(preds, speed)

In [ ]:
show_preds(preds)

In [ ]:

def apply_step(params, prn=True):
    preds = f(time, params)
    loss = mse(preds, speed)
    loss.backward()
    params.data -= lr * params.grad.data
    params.grad = None
    if prn: print(loss.item())
    return preds

In [ ]:
for i in range(10): apply_step(params)

In [ ]:
params = orig_params.detach().requires_grad_()

In [ ]:
_,axs = plt.subplots(1,4,figsize=(12,3))
for ax in axs: show_preds(apply_step(params, False), ax)
plt.tight_layout()

# MNIST loss function

In [ ]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)

In [ ]:
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_x.shape,train_y.shape

In [ ]:
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [ ]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [ ]:
weights = init_params((28*28,1))

In [ ]:
bias = init_params(1)

In [ ]:
(train_x[0]*weights.T).sum() + bias

In [ ]:
def linear1(xb): return xb@weights + bias
preds = linear1(train_x)
preds

In [ ]:
corrects = (preds> 0.0).float() == train_y
corrects

In [ ]:
corrects.float().mean().item()

In [ ]:
weights.data *= 1.03

In [ ]:
preds = linear1(train_x)
((preds>0.0).float() == train_y).float().mean().item()

In [ ]:
trgts = tensor([1, 0, 1])
prds = tensor([0.9, 0.4, 0.2])

In [ ]:
def mnist_loss(predictions, targets):
  return torch.where(targets==1, 1- predictions, predictions).mean()

In [ ]:
torch.where(trgts==1, 1-prds, prds)

In [ ]:
mnist_loss(prds, trgts)

In [ ]:
mnist_loss(tensor([0.9, 0.4, 0.8]), trgts)

# Sigmoid

In [ ]:
def sigmoid(x): return 1/ (1+ torch.exp(-x))

In [ ]:
plot_function(torch.sigmoid, title='Sigmoid', min= -8, max= 8)

In [ ]:
def mnist_loss(predictions, targets):
  predictions = predictions.sigmoid()
  return torch.where(targets == 1, 1-predictions, predictions).mean()

#  SGD and mini-batches

In [ ]:
coll = range(15)
dl = DataLoader(coll, batch_size= 5, shuffle= True)
list(dl)

In [ ]:
ds = L(enumerate(string.ascii_lowercase))
ds

In [ ]:
dl = DataLoader(ds, batch_size= 8, shuffle=True)
list(dl)

In [ ]:
weights = init_params((28*28, 1))
bias = init_params(1)

In [ ]:
dl = DataLoader(dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape

In [ ]:
valid_dl = DataLoader(valid_dset, batch_size=256)

In [ ]:
batch = train_x[:4]
batch.shape

In [ ]:
preds = linear1(batch)
preds

In [ ]:
loss = mnist_loss(preds, train_y[:4])
loss

In [ ]:
loss.backward()
weights.grad.shape, weights.grad.mean(), bias.grad

In [ ]:
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss.backward()

In [ ]:
calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(), bias.grad

In [ ]:
calc_grad(batch, train_y[:4], linear1)
weights.grad.mean(), bias.grad

In [ ]:
weights.grad.zero_()
bias.grad.zero_()

In [ ]:
def train_epoch(model, lr, params):
  for xb, yb in dl:
    calc_grad(xb, yb, model)
    for p in params:
      p.data -= p.grad* lr
      p.grad.zero_()

In [ ]:
def batch_accuracy(xb, yb):
  preds = xb.sigmoid()
  correct = (preds > 0.5) == yb
  return correct.float().mean()

In [ ]:
batch_accuracy(linear1(batch), train_y[:4])

In [ ]:
def validate_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
  return round(torch.stack(accs).mean().item(), 4)

In [ ]:
validate_epoch(linear1)

In [ ]:
lr = 1.
params = weights, bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

In [ ]:
for i in range(20):
  train_epoch(linear1, lr, params)
  print(validate_epoch(linear1), end= ' ')

# Create an Optimizer

In [ ]:
linear_model = nn.Linear(28*28, 1)

In [ ]:
w, b = linear_model.parameters()
w.shape, b.shape

In [ ]:
class BasicOptim:
  def __init__(self, params, lr): self.params, self.lr = list(params), lr

  def step(self, *args, **kwargs):
    for p in self.params: p.data -= p.grad.data * self.lr
  
  def zero_grad(self, *args, **kwargs):
    for p in self.params: p.grad = None

In [ ]:
opt = BasicOptim(linear_model.parameters(), lr)

In [ ]:
def train_epoch(model):
  for xb, yb in dl:
    calc_grad(xb, yb, model)
    opt.step()
    opt.zero_grad()

In [ ]:
validate_epoch(linear_model)

In [ ]:
def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end= ' ')

In [ ]:
train_model(linear_model, 20)

In [ ]:
linear_model = nn.Linear(28*28, 1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(10, lr=lr)

# Adding a non-linearity

In [ ]:
def simple_net(xb):
  res = xb@w1 + b1
  res = res.max(tensor(0.0))
  res = res@w2 + b2
  return res

In [ ]:
w1 = init_params((28* 28, 30))
b1 = init_params(30)
w2 = init_params((30, 1))
b2 = init_params(1)

In [ ]:
plot_function(F.relu)

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28* 28, 30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

In [ ]:
learn = Learner(dls, simple_net, opt_func= SGD, 
                loss_func= mnist_loss, metrics= batch_accuracy)

In [ ]:
learn.fit(40, 0.1)

In [ ]:
plt.plot(L(learn.recorder.values).itemgot(2))

In [ ]:
learn.recorder.values[-1][2]

In [ ]:
learn.model

In [ ]:
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False,
                    loss_func= F.cross_entropy, metrics= accuracy)
learn.fit_one_cycle(1, 0.1)

# Deep learning vocabulary
  - ReLU
    - Function that returns 0 for negative numbers and doesn't change positive numbers.
  - Mini-batch
    - A small group of inputs and labels gathered together in two arrays. A gradient descent step is updated on this batch (rather than a whole epoch).
  - Forward pass
    - Applying the model to some input and computing the predictions.
  - Loss 
    - A value that represents how well (or badly) our model is doing.
  - Gradient 
    - The derivative of the loss with respect to some parameter of the model.
  - Backward pass
    - Computing the gradients of the loss with respect to all model parameters.
  - Gradient descent
    - Taking a step in the directions opposite to the gradients to make the model parameters a little bit better.
  - Learning rate
    - The size of the step we take when applying SGD to update the parameters of the model.